In [166]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [167]:
# Set view options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [168]:
obs = pd.read_csv("../data/obs_windows/observation_24H_prediction_24H_48H_72H.csv")

C:\Users\Conrad\AppData\Local\Temp\ipykernel_21460\3225533299.py:1: DtypeWarning: Columns (3,4,7,8,10,11,12,13,14,15,16,17,19,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  obs = pd.read_csv("../data/obs_windows/observation_24H_prediction_24H_48H_72H.csv")


### 7d observation window with 24hr prediction window

In [169]:
obs_24 = obs.drop(obs.loc[:, 'spn':'full_derate_within_24hr'], axis=1).drop(obs.loc[:, 'partial_derate_within_48hr':'derate_within_72hr'], axis=1).drop(columns='EventTimeStamp')

In [170]:
eq_ids = obs_24.drop_duplicates(subset='EquipmentID')
eq_ids = eq_ids[['EquipmentID']]
eq_ids.shape[0]

1039

In [171]:
np.random.seed(321)

eq_ids['random'] = (np.random.randint(0, 10000, eq_ids.shape[0]))/10000

In [172]:
#create column 'modeling' labeling ~35% of the data for training, ~30% for validation, and ~35% for testing 
eq_ids['modeling'] = np.where(((eq_ids.random <= 0.35)), 'training', np.where(((eq_ids.random <= 0.65)), 'validation', 'testing'))

In [173]:
#merge modeling column onto original dataframe
obs_24 = obs_24.sort_values(by='EquipmentID')
eq_ids = eq_ids.sort_values(by='EquipmentID')

obs_24 = obs_24.merge(eq_ids, on='EquipmentID')
obs_24 = obs_24.drop(columns='EquipmentID')

In [174]:
#creating training, testing, and validation dfs
obs_24_training = obs_24[obs_24['modeling']=='training']
obs_24_training = obs_24_training.drop(columns=['modeling', 'random'])
print(obs_24_training.shape)

obs_24_test = obs_24[obs_24['modeling']=='testing']
obs_24_test = obs_24_test.drop(columns=['modeling', 'random'])
print(obs_24_test.shape)

obs_24_validation = obs_24[obs_24['modeling']=='validation']
obs_24_validation = obs_24_validation.drop(columns=['modeling', 'random'])
print(obs_24_validation.shape)

(200860, 56)
(202749, 56)
(151246, 56)


In [175]:
df_sampling = obs_24_training
df_sampling_non_tenth = df_sampling[df_sampling['derate_within_24hr'] == 0][0:18855]

In [176]:
df_sampling_derate_tenth = pd.concat([df_sampling[df_sampling['derate_within_24hr']==1], df_sampling_non_tenth])

In [177]:
oversampler = SMOTE(k_neighbors=5, random_state=321)

features = df_sampling_derate_tenth.loc[:, 'Aftertreatment System':].columns.tolist()

target = ['derate_within_24hr']

X_train = df_sampling_derate_tenth[features]
y_train = df_sampling_derate_tenth[target]

In [178]:
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

In [179]:
smote_all = pd.concat([y_smote, X_smote], axis=1)
smote_derate = smote_all[smote_all['derate_within_24hr']==1]
smote_derate.shape

(18855, 56)

In [180]:
und_samp = pd.concat([smote_derate, df_sampling[df_sampling['derate_within_24hr']==0]])
und_samp.shape

(215061, 56)

In [181]:
undersampler = RandomUnderSampler(random_state=321, sampling_strategy=1/2)

features = und_samp.loc[:, 'Aftertreatment System':].columns.tolist()

target =['derate_within_24hr']

X_train_final = und_samp[features]
y_train_final = und_samp[target]

X_test = obs_24_test[features]
y_test = obs_24_test[target]

In [182]:
X_resampled, y_resampled = undersampler.fit_resample(X_train_final, y_train_final)

In [183]:
print(y_resampled.shape)

(56565, 1)


In [184]:
rf_pipeline = Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('rf_clf', RandomForestClassifier(n_estimators = 500, n_jobs = -1, random_state = 69))
    ]
)

In [185]:
rand_forest = rf_pipeline.fit(X_resampled, y_resampled)
rf_pipeline.score(X_test, y_test)

y_pred = rand_forest.predict(X_test)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [186]:
data = {'feature': np.array(X_resampled.columns), 'importance': rf_pipeline.named_steps['rf_clf'].feature_importances_}
pd.DataFrame(data).sort_values('importance', ascending = False)


,feature,importance
29,Engine Protection Torque Partial Derate,3.643280e-01
0,Aftertreatment System,2.632842e-01
51,Unknown,1.185657e-01
18,Engine Coolant,5.343279e-02
22,Engine Fuel,3.352403e-02
28,Engine Protection Torque Full Derate,2.595019e-02
41,Intake Manifold,2.076672e-02
43,J1939 Network,1.898140e-02
52,Variable Geometry Turbocharger,1.858892e-02
17,Engine Control Module,1.307703e-02


In [187]:
confusion_matrix(y_test, y_pred)

array([[189828,   8184],
       [  1216,   3521]], dtype=int64)

In [188]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [189]:
(tn, fp, fn, tp)

(189828, 8184, 1216, 3521)

In [190]:
print(classification_report(y_test, y_pred, zero_division = 0))

              precision    recall  f1-score   support

         0.0       0.99      0.96      0.98    198012
         1.0       0.30      0.74      0.43      4737

    accuracy                           0.95    202749
   macro avg       0.65      0.85      0.70    202749
weighted avg       0.98      0.95      0.96    202749



In [191]:
lg_pipeline =  Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('lg_clf', LogisticRegression())
    ]
)

In [192]:
logistic = lg_pipeline.fit(X_resampled, y_resampled)

y_pred2 = logistic.predict(X_test)

y_pred_proba = logistic.predict_proba(X_test)[::,1]

auc = metrics.roc_auc_score(y_test, y_pred_proba)

print(auc)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9029837149132336


### 48hr prediction window

In [193]:
obs_48 = obs.drop(obs.loc[:, 'spn':'full_derate_within_48hr'], axis=1).drop(obs.loc[:, 'partial_derate_within_72hr':'derate_within_72hr'], axis=1).drop(columns='EventTimeStamp')

In [194]:
eq_ids = obs_48.drop_duplicates(subset='EquipmentID')
eq_ids = eq_ids[['EquipmentID']]
eq_ids.shape[0]

1039

In [195]:
np.random.seed(321)

eq_ids['random'] = (np.random.randint(0, 10000, eq_ids.shape[0]))/10000

In [196]:
#create column 'modeling' labeling ~35% of the data for training, ~30% for validation, and ~35% for testing 
eq_ids['modeling'] = np.where(((eq_ids.random <= 0.35)), 'training', np.where(((eq_ids.random <= 0.65)), 'validation', 'testing'))

In [197]:
#merge modeling column onto original dataframe
obs_48 = obs_48.sort_values(by='EquipmentID')
eq_ids = eq_ids.sort_values(by='EquipmentID')

obs_48 = obs_48.merge(eq_ids, on='EquipmentID')
obs_48 = obs_48.drop(columns='EquipmentID')

In [198]:
#creating training, testing, and validation dfs
obs_48_training = obs_48[obs_48['modeling']=='training']
obs_48_training = obs_48_training.drop(columns=['modeling', 'random'])
print(obs_48_training.shape)

obs_48_test = obs_48[obs_48['modeling']=='testing']
obs_48_test = obs_48_test.drop(columns=['modeling', 'random'])
print(obs_48_test.shape)

obs_48_validation = obs_48[obs_48['modeling']=='validation']
obs_48_validation = obs_48_validation.drop(columns=['modeling', 'random'])
print(obs_48_validation.shape)

(200860, 56)
(202749, 56)
(151246, 56)


In [199]:
df_sampling = obs_48_training
df_sampling_non_tenth = df_sampling[df_sampling['derate_within_48hr'] == 0][0:18855]

In [200]:
df_sampling_derate_tenth = pd.concat([df_sampling[df_sampling['derate_within_48hr']==1], df_sampling_non_tenth])

In [201]:
oversampler = SMOTE(k_neighbors=5, random_state=321)

features = df_sampling_derate_tenth.loc[:, 'Aftertreatment System':].columns.tolist()

target = ['derate_within_48hr']

X_train = df_sampling_derate_tenth[features]
y_train = df_sampling_derate_tenth[target]

X_test = obs_48_test[features]
y_test = obs_48_test[target]

In [202]:
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

In [203]:
smote_all = pd.concat([y_smote, X_smote], axis=1)
smote_derate = smote_all[smote_all['derate_within_48hr']==1]
smote_derate.shape

(18855, 56)

In [204]:
und_samp = pd.concat([smote_derate, df_sampling[df_sampling['derate_within_48hr']==0]])
und_samp.shape

(214184, 56)

In [205]:
undersampler = RandomUnderSampler(random_state=321, sampling_strategy=1/2)

features = und_samp.loc[:, 'Aftertreatment System':].columns.tolist()

target =['derate_within_48hr']

X_train_final = und_samp[features]
y_train_final = und_samp[target]

In [206]:
X_resampled, y_resampled = undersampler.fit_resample(X_train_final, y_train_final)

In [207]:
print(y_resampled.shape)

(56565, 1)


In [208]:
rf_pipeline = Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('rf_clf', RandomForestClassifier(n_estimators = 500, n_jobs = -1, random_state = 69))
    ]
)

In [209]:
rand_forest = rf_pipeline.fit(X_resampled, y_resampled)
rf_pipeline.score(X_test, y_test)

y_pred = rand_forest.predict(X_test)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [210]:
data = {'feature': np.array(X_resampled.columns), 'importance': rf_pipeline.named_steps['rf_clf'].feature_importances_}
pd.DataFrame(data).sort_values('importance', ascending = False)


,feature,importance
29,Engine Protection Torque Partial Derate,3.481385e-01
0,Aftertreatment System,2.581258e-01
51,Unknown,1.258513e-01
18,Engine Coolant,5.255591e-02
22,Engine Fuel,3.559551e-02
28,Engine Protection Torque Full Derate,2.413300e-02
43,J1939 Network,2.357732e-02
17,Engine Control Module,1.855276e-02
52,Variable Geometry Turbocharger,1.840694e-02
41,Intake Manifold,1.803552e-02


In [211]:
confusion_matrix(y_test, y_pred)

array([[189105,   8223],
       [  1657,   3764]], dtype=int64)

In [212]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [213]:
(tn, fp, fn, tp)

(189105, 8223, 1657, 3764)

In [214]:
print(classification_report(y_test, y_pred, zero_division = 0))

              precision    recall  f1-score   support

         0.0       0.99      0.96      0.97    197328
         1.0       0.31      0.69      0.43      5421

    accuracy                           0.95    202749
   macro avg       0.65      0.83      0.70    202749
weighted avg       0.97      0.95      0.96    202749



In [215]:
lg_pipeline =  Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('lg_clf', LogisticRegression())
    ]
)

In [216]:
logistic = lg_pipeline.fit(X_resampled, y_resampled)

y_pred2 = logistic.predict(X_test)

y_pred_proba = logistic.predict_proba(X_test)[::,1]

auc = metrics.roc_auc_score(y_test, y_pred_proba)

print(auc)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8721184780559064


### 7d observation window with 72hr prediction window

In [217]:
obs_72 = obs.drop(obs.loc[:, 'spn':'full_derate_within_72hr'], axis=1).drop(columns='EventTimeStamp')

In [218]:
eq_ids = obs_72.drop_duplicates(subset='EquipmentID')
eq_ids = eq_ids[['EquipmentID']]
eq_ids.shape[0]

1039

In [219]:
np.random.seed(321)

eq_ids['random'] = (np.random.randint(0, 10000, eq_ids.shape[0]))/10000

In [220]:
#create column 'modeling' labeling ~35% of the data for training, ~30% for validation, and ~35% for testing 
eq_ids['modeling'] = np.where(((eq_ids.random <= 0.35)), 'training', np.where(((eq_ids.random <= 0.65)), 'validation', 'testing'))

In [221]:
#merge modeling column onto original dataframe
obs_72 = obs_72.sort_values(by='EquipmentID')
eq_ids = eq_ids.sort_values(by='EquipmentID')

obs_72 = obs_72.merge(eq_ids, on='EquipmentID')
obs_72 = obs_72.drop(columns='EquipmentID')

In [222]:
#creating training, testing, and validation dfs
obs_72_training = obs_72[obs_72['modeling']=='training']
obs_72_training = obs_72_training.drop(columns=['modeling', 'random'])
print(obs_72_training.shape)

obs_72_test = obs_72[obs_72['modeling']=='testing']
obs_72_test = obs_72_test.drop(columns=['modeling', 'random'])
print(obs_72_test.shape)

obs_72_validation = obs_72[obs_72['modeling']=='validation']
obs_72_validation = obs_72_validation.drop(columns=['modeling', 'random'])
print(obs_72_validation.shape)

(200860, 56)
(202749, 56)
(151246, 56)


In [223]:
df_sampling = obs_72_training
df_sampling_non_tenth = df_sampling[df_sampling['derate_within_72hr'] == 0][0:18855]

In [224]:
df_sampling_derate_tenth = pd.concat([df_sampling[df_sampling['derate_within_72hr']==1], df_sampling_non_tenth])

In [225]:
oversampler = SMOTE(k_neighbors=5, random_state=321)

features = df_sampling_derate_tenth.loc[:, 'Aftertreatment System':].columns.tolist()

target = ['derate_within_72hr']

X_train = df_sampling_derate_tenth[features]
y_train = df_sampling_derate_tenth[target]

In [226]:
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

In [227]:
smote_all = pd.concat([y_smote, X_smote], axis=1)
smote_derate = smote_all[smote_all['derate_within_72hr']==1]
smote_derate.shape

(18855, 56)

In [228]:
und_samp = pd.concat([smote_derate, df_sampling[df_sampling['derate_within_72hr']==0]])
und_samp.shape

(213477, 56)

In [229]:
undersampler = RandomUnderSampler(random_state=321, sampling_strategy=1/2)

features = und_samp.loc[:, 'Aftertreatment System':].columns.tolist()

target =['derate_within_72hr']

X_train_final = und_samp[features]
y_train_final = und_samp[target]

X_test = obs_72_test[features]
y_test = obs_72_test[target]

In [230]:
X_resampled, y_resampled = undersampler.fit_resample(X_train_final, y_train_final)

In [231]:
print(y_resampled.shape)

(56565, 1)


In [232]:
rf_pipeline = Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('rf_clf', RandomForestClassifier(n_estimators = 500, n_jobs = -1, random_state = 69))
    ]
)

In [233]:
rand_forest = rf_pipeline.fit(X_resampled, y_resampled)
rf_pipeline.score(X_test, y_test)

y_pred = rand_forest.predict(X_test)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [234]:
data = {'feature': np.array(X_resampled.columns), 'importance': rf_pipeline.named_steps['rf_clf'].feature_importances_}
pd.DataFrame(data).sort_values('importance', ascending = False)


,feature,importance
29,Engine Protection Torque Partial Derate,0.337164
0,Aftertreatment System,0.266651
51,Unknown,0.124600
18,Engine Coolant,0.052711
22,Engine Fuel,0.035147
28,Engine Protection Torque Full Derate,0.025482
43,J1939 Network,0.023277
41,Intake Manifold,0.020639
17,Engine Control Module,0.019024
52,Variable Geometry Turbocharger,0.016404


In [235]:
confusion_matrix(y_test, y_pred)

array([[187925,   8926],
       [  1924,   3974]], dtype=int64)

In [236]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [237]:
(tn, fp, fn, tp)

(187925, 8926, 1924, 3974)

In [238]:
print(classification_report(y_test, y_pred, zero_division = 0))

              precision    recall  f1-score   support

         0.0       0.99      0.95      0.97    196851
         1.0       0.31      0.67      0.42      5898

    accuracy                           0.95    202749
   macro avg       0.65      0.81      0.70    202749
weighted avg       0.97      0.95      0.96    202749



In [239]:
lg_pipeline =  Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('lg_clf', LogisticRegression())
    ]
)

In [240]:
logistic = lg_pipeline.fit(X_resampled, y_resampled)

y_pred2 = logistic.predict(X_test)

y_pred_proba = logistic.predict_proba(X_test)[::,1]

auc = metrics.roc_auc_score(y_test, y_pred_proba)

print(auc)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8568800134172223
